In [1]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join('..','./data/', './clean-data/')))
sys.path.append(os.path.abspath(os.path.join('..','./scripts/')))

# Load the dataset into a pandas DataFrame
df = pd.read_csv(Path("../data/clean-data/","predicted_co2_rating.csv"))



## To do: clustering fuel only cars
1. T-SNE, read about it, do dimensionality reduction

## Pair wise clustering
1. Hybrid and electric
2. Hybrid and fuel

## Fuzzy logic
Can we calculate car membership?
Read on fuzzy logic 

## Consumer analysis with scraped data 
Predictor for time series sales data
Couple sales data with the co2 scores
Accumulated prediction for co2 impact 


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26961 entries, 0 to 26960
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   vehicleclass_                  26961 non-null  object 
 1   make_                          26961 non-null  object 
 2   model.1_                       26961 non-null  object 
 3   model_year                     26961 non-null  int64  
 4   cylinders_                     26961 non-null  float64
 5   fuelconsumption_city(l/100km)  26961 non-null  float64
 6   fuelconsumption_hwy(l/100km)   26961 non-null  float64
 7   fuelconsumption_comb(l/100km)  26961 non-null  float64
 8   co2emissions_(g/km)            26961 non-null  int64  
 9   number_of_gears                26961 non-null  float64
 10  predicted_co2_rating           26961 non-null  int64  
 11  original_co2r                  7973 non-null   float64
 12  enginesize_(l)                 26961 non-null 

In [ ]:
var_list = ['vehicleclass_','make_',
                    'model.1_','model_year',
                    'cylinders_','fuelconsumption_city(l/100km)',
                    'fuelconsumption_hwy(l/100km)',
                    'fuelconsumption_comb(l/100km)',
                    'predicted_co2_rating',
                    'co2emissions_(g/km)',
                    'number_of_gears']


# Set up parameters for the model - numerical and categorical
numeric_features =  ['model_year','cylinders_',
                    'fuelconsumption_city(l/100km)',
                    'fuelconsumption_hwy(l/100km)',
                    'fuelconsumption_comb(l/100km)',
                    'predicted_co2_rating',
                    'co2emissions_(g/km)','number_of_gears']
categorical_features = ['vehicleclass_']

# Set up numerical and categorical transformers
numeric_transformer = Pipeline(
                            steps=[("scaler", StandardScaler())]
                        )

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Set up preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        #("cat", categorical_transformer, categorical_features),
    ]
)


In [7]:
# Preprocess the data by removing irrelevant columns and normalizing the features
df = df[['model_year', 'cylinders_', 'number_of_gears','co2emissions_(g/km)', 'predicted_co2_rating','enginesize_(l)']]
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Fit the KMeans algorithm to the scaled data
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_scaled)

# Predict the cluster for each row in the dataset
df['cluster'] = kmeans.predict(df_scaled)

C:\Users\Laura GF\AppData\Local\Temp\ipykernel_6028\1783735641.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = kmeans.predict(df_scaled)
